In [5]:
!pwd

/Users/tiagocunha/Documents/Jupyter


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import logging

In [7]:
logging.basicConfig(level=logging.INFO)

In [8]:
def create_model():
    model = Sequential()
    model.add(Dense(20, activation='relu', input_dim=204))
    model.add(Dense(4, activation='softmax')) # FOUR OUTPUTS ARE EXPECTED
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
np.random.seed(42)

In [10]:
data = pd.read_excel('/Users/tiagocunha/Documents/workspace/data/apples.xlsx', engine='openpyxl')

In [11]:
data.head()

,Band1,Band2,Band3,Band4,Band5,Band6,Band7,Band8,Band9,Band10,...,Band200,Band201,Band202,Band203,Band204,Label,Class 0,Class 1,Class 2,Class 3
0,0.366667,0.333333,0.28,0.253968,0.240000,0.218391,0.222222,0.207207,0.203252,0.207407,...,0.583333,0.553571,0.584906,0.58,0.586957,0,1,0,0,0
1,0.400000,0.282051,0.28,0.238095,0.226667,0.195402,0.202020,0.198198,0.195122,0.185185,...,0.566667,0.571429,0.584906,0.58,0.608696,0,1,0,0,0
2,0.366667,0.307692,0.26,0.222222,0.213333,0.195402,0.191919,0.189189,0.178862,0.170370,...,0.550000,0.571429,0.584906,0.58,0.565217,0,1,0,0,0
3,0.366667,0.307692,0.24,0.222222,0.200000,0.195402,0.171717,0.180180,0.154472,0.155556,...,0.550000,0.571429,0.566038,0.56,0.586957,0,1,0,0,0
4,0.366667,0.307692,0.26,0.206349,0.200000,0.183908,0.181818,0.162162,0.154472,0.148148,...,0.600000,0.589286,0.584906,0.60,0.622222,0,1,0,0,0


In [8]:

dfX = data[[f'Band{i}' for i in range(1,205)]]
# dfY = data['Label']
dfY = data[['Class 0', 'Class 1', 'Class 2', 'Class 3']]

In [9]:

print(dfX.shape)
print(dfY.shape)

(143775, 204)
(143775, 4)


In [10]:
X = dfX.to_numpy()
y = dfY.to_numpy()



In [11]:
clf = KerasClassifier(build_fn=create_model, epochs=10, verbose=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
clf.fit(X_train, y_train)

Epoch 1/10
2247/2247 [==============================] - 25s 1ms/step - loss: 0.2857 - accuracy: 0.9050
Epoch 2/10
2247/2247 [==============================] - 3s 1ms/step - loss: 0.0266 - accuracy: 0.9906
Epoch 3/10
2247/2247 [==============================] - 3s 1ms/step - loss: 0.0207 - accuracy: 0.9925
Epoch 4/10
2247/2247 [==============================] - 3s 1ms/step - loss: 0.0184 - accuracy: 0.9927
Epoch 5/10
2247/2247 [==============================] - 4s 2ms/step - loss: 0.0194 - accuracy: 0.9928
Epoch 6/10
2247/2247 [==============================] - 4s 2ms/step - loss: 0.0186 - accuracy: 0.9933
Epoch 7/10
2247/2247 [==============================] - 4s 2ms/step - loss: 0.0179 - accuracy: 0.9937
Epoch 8/10
2247/2247 [==============================] - 5s 2ms/step - loss: 0.0164 - accuracy: 0.9936
Epoch 9/10
2247/2247 [==============================] - 4s 2ms/step - loss: 0.0162 - accuracy: 0.9941: 0s - loss: 0.0161 - 
Epoch 10/10
2247/2247 [==============================] - 3s

In [12]:
y_pred = clf.predict(X_test)
print(y_pred)

/Users/tiagocunha/opt/anaconda3/envs/spyder-env/lib/python3.8/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


2247/2247 [==============================] - 3s 1ms/step
[3 3 2 ... 2 1 1]


In [13]:
print(f"Test Score: {clf.score(X_test, y_test)}")

2247/2247 [==============================] - 4s 2ms/step - loss: 0.0188 - accuracy: 0.9930
Test Score: 0.9930030107498169


In [14]:
import tensorflow as tf

In [15]:
# Convert the model.
clf.model.save('apples_model')


INFO:tensorflow:Assets written to: apples_model/assets


INFO:tensorflow:Assets written to: apples_model/assets


In [68]:

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 204)
      yield [data.astype(np.float32)]

# converter = tf.lite.TFLiteConverter.from_keras_model(clf.model)
converter = tf.lite.TFLiteConverter.from_saved_model('apples_model')
#Quantize the model
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS] # enable TensorFlow Lite ops.]

#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.representative_dataset = representative_dataset

#converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

# Save the model.
with open('apples_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [91]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str  += '#ifndef ' +  var_name.upper() +  '_H\n'
  c_str  += '#define ' +  var_name.upper() +  '_H\n\n'

  # Add array length at top of file
  c_str  += '\nunsigned int ' +  var_name +  '_len = ' +  str(len(hex_data)) +  ';\n'

  # Declare C variable
  c_str += 'unsigned char '  + var_name  + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i +  1) < len(hex_data):
      hex_str += ','
    if (i +  1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' +  format(' '.join(hex_array)) +  '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' +  var_name.upper() +  '_H'

  return c_str

In [92]:
# Write TFLite model to a C source (or header) file
with open('apples_model.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, 'apples_model'))

In [62]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter('apples_model.tflite')

In [66]:
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
#print(input_data)
input_data = np.array([[0.366666675, 0.333333343, 0.280000001, 0.253968269, 0.239999995, 0.218390808, 0.222222224, 0.207207203, 0.203252032, 0.2074074, 0.201342285, 0.196319014, 0.196629211, 0.195876285, 0.193396226, 0.192139745, 0.19758065, 0.200000003, 0.199312717, 0.203821659, 0.207715139, 0.207282916, 0.206349209, 0.208542719, 0.213942304, 0.211009175, 0.217105269, 0.220588237, 0.218875498, 0.219230771, 0.22324723, 0.224778756, 0.225255966, 0.22660099, 0.231378764, 0.2324159, 0.232352942, 0.230878189, 0.234652117, 0.231979027, 0.237074405, 0.240875915, 0.238823533, 0.242009133, 0.245283023, 0.24406047, 0.243414119, 0.245901644, 0.246246248, 0.24583742, 0.248803824, 0.248826295, 0.25, 0.250678748, 0.251561105, 0.253744483, 0.253472209, 0.25731498, 0.257021278, 0.255480617, 0.259631485, 0.2570481, 0.263201326, 0.260869563, 0.268074721, 0.268391281, 0.271922767, 0.27635783, 0.283439487, 0.288095236, 0.29588607, 0.302454472, 0.308846772, 0.314624518, 0.324345767, 0.334126979, 0.344470948, 0.351718634, 0.362690151, 0.372485936, 0.387278587, 0.394842863, 0.40647772, 0.418092906, 0.419196069, 0.422757477, 0.427135676, 0.429661006, 0.439862549, 0.439722449, 0.441512764, 0.439678282, 0.431425989, 0.42949906, 0.423984885, 0.426923066, 0.429268301, 0.436023623, 0.459325403, 0.493506491, 0.527472556, 0.561999977, 0.590772331, 0.611222446, 0.622983873, 0.636548221, 0.642126799, 0.653526962, 0.656512618, 0.660638273, 0.664502144, 0.668859661, 0.667781472, 0.674259663, 0.670520246  ,0.666666687  ,0.667067289, 0.663003683, 0.669987559, 0.662420392, 0.665799737, 0.667110503, 0.672554374, 0.667593896, 0.667617679, 0.673469365, 0.670640826, 0.666666687, 0.672386885, 0.674679458, 0.683774829, 0.666666687, 0.674782634, 0.673796773, 0.669104218, 0.67608285, 0.680154145, 0.672619045, 0.675510228, 0.68058455, 0.672376871, 0.676211476, 0.669662893, 0.673563242, 0.665882349, 0.669879496, 0.665841579, 0.653164566, 0.664935052, 0.65691489, 0.657608688, 0.65459609, 0.658119678, 0.656976759, 0.658753693, 0.669696987, 0.663580239, 0.661392391, 0.655844152, 0.665551841, 0.656357408, 0.657243788, 0.649819493, 0.650557637, 0.664122164, 0.64453125, 0.641129017, 0.647302926, 0.648068666, 0.64159292, 0.645454526, 0.63380283, 0.631067932, 0.623115599, 0.625, 0.618279576, 0.611111104, 0.614942551, 0.613095224, 0.602484465, 0.597402573, 0.587837815, 0.598591566, 0.602941155, 0.592307687, 0.59677422, 0.593220353, 0.566371679, 0.570093453, 0.57843136, 0.551020384, 0.569892466, 0.556818187, 0.559523821, 0.556962013, 0.560000002, 0.577464759, 0.567164183, 0.578125, 0.583333313, 0.553571403, 0.584905684, 0.579999983, 0.586956501]], dtype=np.float32)
#print(input_details[0]['index'])
interpreter.set_tensor(input_details[0]['index'], input_data)
#print(input_details[0].keys())
#print(output_details)
#print(input_shape)
#print(input_data.shape)
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.99609375 0.00390625 0.         0.        ]]


In [2]:
labels = ["real red apple", "fake red apple" , "real green apple", "fake green apple"]
print(f"Most probable class: {label[np.argmax(output_data)] if np.any(output_data>0.8) else None}")

<IPython.core.display.Javascript object>

NameError: name 'output_data' is not defined